In [1]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import glob
import cv2

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
#     print(len(ret))
    for i in range(len(ret)):
#         if i % 20 == 0:
#             print(i)
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (224, 224), cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    print('Processed yet another image')
    return [path, resized]

def normalize_image_features(paths):
    print('here')
    imf_d = {}
    p = Pool(cpu_count())
    print('here_1')
    ret = p.map(get_im_cv2, paths)
    print(len(ret))
    for i in range(len(ret)):
        if i % 20 == 0:
            print(i)        
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
#     fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

def subtract_mean(data, mean):
    for sample in data:
        sample - mean
    return data


In [3]:
train =  glob.glob('/mnt/train/**/*.jpg') + glob.glob('/mnt/additional/**/*.jpg')
train = pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train], columns = ['type','image','path'])
print(train.shape)

(8208, 3)


In [4]:
train = glob.glob('/mnt/train/**/*.jpg') + glob.glob('/mnt/additional/**/*.jpg')
train = pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train], columns = ['type','image','path'])
train = im_stats(train)
train = train[train['size'] != '0 0'].reset_index(drop=True) #corrupt images removed
print("Bad images removed")
print("loading train data")
train_data = normalize_image_features(train['path'])

print('sustracting mean')
mean = train_data.mean(axis=0)
train_data = subtract_mean(train_data, mean)

print("train data loaded")
np.save('/mnt/additional/train_vgg_addi.npy', train_data, allow_pickle=True, fix_imports=True)

print("Preparing target")
le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
print(le.classes_) 
np.save('/mnt/additional/train_target_vgg_addi.npy', train_target, allow_pickle=True, fix_imports=True)

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 393216 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 33554432 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly c

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 25165824 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205389824 bytes but only got 0. Skipping tag 5
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 486014976 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Poss

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 12429. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 701300736 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3640590336 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 99877888 bytes but only got 0. Skipping tag 1029
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWa

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 9628. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 10149. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 853671936 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 634126336 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWa

/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 205914112 bytes but only got 10037. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 450494464 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 651427840 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 515309568 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/mwrln/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarni

...

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
8205
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
38

Process ForkPoolWorker-2:
Process ForkPoolWorker-12:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Process ForkPoolWorker-7:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/p

In [3]:
train_data = np.load('/mnt/additional/train_vgg_addi.npy')
mean = train_data.mean(axis=0)

In [4]:
test = glob.glob('test/*.jpg')
test = pd.DataFrame([[p.split('/')[1],p] for p in test], columns = ['image','path']) 
print(test.shape)
print(test.head())

test_data = normalize_image_features(test['path'])

print('Subtracting mean')
test_data = subtract_mean(test_data, mean)

np.save('test_vgg_addi.npy', test_data, allow_pickle=True, fix_imports=True)
print("test data loaded")
test_id = test.image.values
np.save('test_id_vgg_addi.npy', test_id, allow_pickle=True, fix_imports=True)

(512, 2)
     image          path
0  351.jpg  test/351.jpg
1  132.jpg  test/132.jpg
2  275.jpg  test/275.jpg
3  481.jpg  test/481.jpg
4  452.jpg  test/452.jpg
here
here_1
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet ano

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

NameError: name 'mean' is not defined

In [4]:
test_2 = glob.glob('/mnt/test_stg2/*.jpg')
test_2 = pd.DataFrame([[p.split('/')[3],p] for p in test_2], columns = ['image','path']) #[::20] #limit for Kaggle Demo
print("loading train data")
test_2_data = normalize_image_features(test_2['path'])

print('Subtractin mean')
test_data = subtract_mean(test_2_data, mean)

np.save('/mnt/additional/test_2_vgg_addi.npy', test_2_data, allow_pickle=True, fix_imports=True)
print("test data loaded")
test_2_id = test_2.image.values
np.save('/mnt/additional/test_2_id_vgg_addi.npy', test_2_id, allow_pickle=True, fix_imports=True)

loading train data
here
here_1
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet ano

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

...

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet another image
Processed yet anothe

Subtractin mean
test data loaded


Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", 

In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.callbacks import ModelCheckpoint

from keras import applications

# K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_last')
K.set_floatx('float32')

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [4]:
from keras.models import Model
from keras import optimizers
np.random.seed(17)

train_data = np.load('/mnt/additional/train_vgg_addi.npy')
train_target = np.load('/mnt/additional/train_target_vgg_addi.npy')

print('Data loaded')
print(train_data.shape)

BATCH_SIZE = 4

vgg16 = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(4096, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(4096, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3, activation='softmax'))

model = Model(input= vgg16.input, output= top_model(vgg16.output))

opt = optimizers.SGD(lr=0.001, momentum=0.9, decay=1e-4, nesterov=False) #optimizers.Nadam(lr=0.0002)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

print('Loaded vgg16')
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

Data loaded
(8205, 224, 224, 3)
Loaded vgg16


/home/mwrln/.local/lib/python3.5/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


In [7]:
np.random.seed(17)

BATCH_SIZE = 4

x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.4, random_state=17)
print("fitting data")
model.fit_generator(datagen.flow(x_train,y_train, batch_size=BATCH_SIZE, shuffle=True), nb_epoch=150, callbacks=[ModelCheckpoint('vgg_model_additional.hdf5', save_best_only=True)], samples_per_epoch=len(x_train), verbose=1, validation_data=(x_val_train, y_val_train))
print("data fitted in model")

fitting data
Epoch 1/150


/home/mwrln/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=(array([[[..., epochs=150, callbacks=[<keras.ca..., verbose=1, steps_per_epoch=1230)`
  import sys


1230/1230 [==============================] - 338s - loss: 0.9220 - acc: 0.5526 - val_loss: 0.8957 - val_acc: 0.5637
Epoch 2/150
1230/1230 [==============================] - 319s - loss: 0.9213 - acc: 0.5513 - val_loss: 0.9504 - val_acc: 0.5469
Epoch 3/150
1230/1230 [==============================] - 337s - loss: 0.9213 - acc: 0.5497 - val_loss: 0.8912 - val_acc: 0.5664
Epoch 4/150
1230/1230 [==============================] - 317s - loss: 0.9207 - acc: 0.5523 - val_loss: 0.8979 - val_acc: 0.5631
Epoch 5/150
1230/1230 [==============================] - 320s - loss: 0.9181 - acc: 0.5580 - val_loss: 0.9060 - val_acc: 0.5652
Epoch 6/150
1230/1230 [==============================] - 317s - loss: 0.9174 - acc: 0.5507 - val_loss: 0.8996 - val_acc: 0.5591
Epoch 7/150
1230/1230 [==============================] - 317s - loss: 0.9159 - acc: 0.5492 - val_loss: 0.9063 - val_acc: 0.5500
Epoch 8/150
1230/1230 [==============================] - 344s - loss: 0.9152 - acc: 0.5547 - val_loss: 0.8877 - val_

1230/1230 [==============================] - 321s - loss: 0.6723 - acc: 0.6989 - val_loss: 0.7982 - val_acc: 0.6533
Epoch 128/150
1230/1230 [==============================] - 340s - loss: 0.6811 - acc: 0.6951 - val_loss: 0.7911 - val_acc: 0.6587
Epoch 129/150
1230/1230 [==============================] - 320s - loss: 0.6808 - acc: 0.6975 - val_loss: 0.7958 - val_acc: 0.6572
Epoch 130/150
1230/1230 [==============================] - 320s - loss: 0.6773 - acc: 0.6988 - val_loss: 0.8624 - val_acc: 0.6328
Epoch 131/150
1230/1230 [==============================] - 322s - loss: 0.6658 - acc: 0.7052 - val_loss: 0.8214 - val_acc: 0.6438
Epoch 132/150
1230/1230 [==============================] - 324s - loss: 0.6608 - acc: 0.7117 - val_loss: 0.8141 - val_acc: 0.6420
Epoch 133/150
1230/1230 [==============================] - 321s - loss: 0.6680 - acc: 0.6982 - val_loss: 0.8250 - val_acc: 0.6356
Epoch 134/150
1230/1230 [==============================] - 321s - loss: 0.6629 - acc: 0.7064 - val_loss:

KeyboardInterrupt: 

In [8]:
from keras.models import load_model
model = load_model('vgg_model_additional.hdf5')

In [9]:
test_data = np.load('test_vgg_addi.npy')
test_id = np.load('test_id_vgg_addi.npy')
print("creating predictions")
predictions = model.predict(test_data)
print("predictions made")

df = pd.DataFrame(predictions, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
# df.to_csv('submission_vgg_new_add.csv', index=False)



creating predictions
predictions made


In [10]:
test_2_data = np.load('/mnt/additional/test_2_vgg_addi.npy')
test_2_id = np.load('/mnt/additional/test_2_id_vgg_addi.npy')
print("creating predictions")
predictions = model.predict(test_2_data)
print("predictions made")

df_1 = pd.DataFrame(predictions, columns=['Type_1','Type_2','Type_3'])
df_1['image_name'] = test_2_id
# df_1.to_csv('submission_vgg_2_new_test2_add.csv', index=False)

creating predictions
predictions made


In [11]:
result = pd.concat([df, df_1])
print(result.shape)
result.head()

(4018, 4)


,Type_1,Type_2,Type_3,image_name
0,0.049029,0.932408,0.018564,351.jpg
1,0.043543,0.448384,0.508073,132.jpg
2,0.268283,0.470911,0.260806,275.jpg
3,0.001222,0.063697,0.935080,481.jpg
4,0.001113,0.179788,0.819098,452.jpg


In [12]:
result.to_csv('subm_vgg_with_addi.csv', index=False)